In [8]:
from keras.models import Sequential, Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import imgaug as ia
from tqdm import tqdm
from imgaug import augmenters as iaa
import numpy as np
import pickle
import os, cv2

In [1]:
LABELS = ['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting',
          'Cyclist', 'Tram', 'Misc', 'DontCare']

LABELS = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
          'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
          'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
          'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
          'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
          'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
          'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
          'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
          'hair drier', 'toothbrush']

LABELS = ['Pedestrian', 'Cyclist', 'DontCare', 'Car', 'Cyclist', 'DontCare', 'Van', 'Tram', 'Truck', 'DontCare', 'DontCare',
          'DontCare', 'DontCare', 'DontCare', 'Person_sitting', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare',
          'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare',
          'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare',
          'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare',
          'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare',
          'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare',
          'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare',
          'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare', 'DontCare',
          'DontCare', 'DontCare']

In [2]:
wt_path = '/Datset/yolov3.weights'
train_image_folder = '/Dataset/image_2'
train_annot_folder = '/Dataset/label_2'
valid_image_folder = '/Dataset/image_2_test'
valid_annot_folder = '/Dataset/label_2_test'

In [3]:
class WeightReader:
    def __init__(self, weight_file):
        with open(weight_file, 'rb') as w_f:
            major,    = struct.unpack('i', w_f.read(4))
            minor,    = struct.unpack('i', w_f.read(4))
            revision, = struct.unpack('i', w_f.read(4))

            if (major*10 + minor) >= 2 and major < 1000 and minor < 1000:
                w_f.read(8)
            else:
                w_f.read(4)

            transpose = (major > 1000) or (minor > 1000)
            
            binary = w_f.read()

        self.offset = 0
        self.all_weights = np.frombuffer(binary, dtype='float32')
        
    def read_bytes(self, size):
        self.offset = self.offset + size
        return self.all_weights[self.offset-size:self.offset]

    def load_weights(self, model):
        for i in range(106):
            try:
                conv_layer = model.get_layer('conv_' + str(i))
                print("loading weights of convolution #" + str(i))

                if i not in [81, 93, 105]:
                    norm_layer = model.get_layer('bnorm_' + str(i))

                    size = np.prod(norm_layer.get_weights()[0].shape)

                    beta  = self.read_bytes(size) # bias
                    gamma = self.read_bytes(size) # scale
                    mean  = self.read_bytes(size) # mean
                    var   = self.read_bytes(size) # variance            

                    weights = norm_layer.set_weights([gamma, beta, mean, var])  

                if len(conv_layer.get_weights()) > 1:
                    bias   = self.read_bytes(np.prod(conv_layer.get_weights()[1].shape))
                    kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
                    
                    kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
                    kernel = kernel.transpose([2,3,1,0])
                    conv_layer.set_weights([kernel, bias])
                else:
                    kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
                    kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
                    kernel = kernel.transpose([2,3,1,0])
                    conv_layer.set_weights([kernel])
            except ValueError:
                print("no convolution #" + str(i))     
    
    def reset(self):
        self.offset = 0
